In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 627, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 627 (delta 132), reused 179 (delta 106), pack-reused 361
Receiving objects: 100% (627/627), 10.74 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (316/316), done.


<br><br><br>

### **part 2 : Motor Trend Car Road Tests**

In [6]:
import pandas as pd

X_train= pd.read_csv('./yemoonsaBigdata/datasets/Part2/mpg_X_train.csv')
X_test= pd.read_csv('./yemoonsaBigdata/datasets/Part2/mpg_X_test.csv')
y_train = pd.read_csv('./yemoonsaBigdata/datasets/Part2/mpg_y_train.csv')

In [7]:
X_train.head(3)

,name,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
0,pontiac j2000 se hatchback,31.0,4,112.0,85.0,2575,16.2,82
1,pontiac safari (sw),13.0,8,400.0,175.0,5140,12.0,71
2,mazda glc custom l,37.0,4,91.0,68.0,2025,18.2,82


In [8]:
X_test.head(3)

,name,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
0,maxda glc deluxe,34.1,4,86.0,65.0,1975,15.2,79
1,plymouth sapporo,23.2,4,156.0,105.0,2745,16.7,78
2,dodge coronet brougham,16.0,8,318.0,150.0,4190,13.0,76


In [9]:
y_train.head(3)

,isUSA
0,1
1,1
2,0


0 : 타지역  
1 : 미국  

<br><br>

#### **데이터 전처리**


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB


<br>

차량 모델 별 고유한 이름을 뜻하는 `name`을 제외한 나머지 column은 모두 float64 혹은 int64로, 

모두 수치형으로 지정되어 있어 별도의 데이터 타입 변경 작업은 필요없을 것으로 보인다.

<br>

따라서 결측치의 비율을 확인하고 처리하는 작업을 진행한다.

결측치를 제거하는 것이 빠르고 쉬운 방법이지만, 채우기 방법을 사용하여 결측치를 처리하도록 하자.

`fillna` 대신에 sklearn의 `impute`를 사용해보자.

In [11]:
X_train.isnull().sum()

name            0
mpg             0
cylinders       0
displacement    0
horsepower      4
weight          0
acceleration    0
model_year      0
dtype: int64

In [12]:
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']] = imputer.fit_transform(X_test[['horsepower']])

<br>

`missing_values` 파라미터는 데이터 상에서 결측치에 해당하는 값을 알려주는 것

`strategy` 파라미터는 결측치를 통계값으로 채워주거나 원하는 상수갑으로 직접 넣을 수도 있다.

- `strategy`를 `constant`로 하고 `fill_value=0` 파라미터를 추가하면 `fillna`와 동일한 동작을 하게 된다.

In [13]:
X_train.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000
mean,23.732734,5.374101,189.994604,103.383212,2948.464029,15.580216,76.057554
std,7.647295,1.677084,105.471423,38.695458,862.949746,2.745907,3.605591
min,10.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,18.000000,4.000000,98.000000,75.000000,2206.250000,14.000000,73.000000
50%,23.000000,4.000000,140.500000,90.500000,2737.500000,15.500000,76.000000
75%,29.000000,6.000000,258.000000,118.750000,3560.000000,17.000000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


<br>

또한 `describe`를 통해 평균, 표준편차, 사분위수 등 기본적인 요약 통계량 값을 확인한다.

이런 정보를 토대로 이상치를 파악하고 대처할 수 있어야 한다.

이번 데이터셋에서는 이상치가 없는 것 같으므로 따로 처리하지 않도록 한다.

<br><br>

In [15]:
remove_columns = ['name']
used_columns = [col for col in X_train.columns if col not in remove_columns]    # ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']

In [19]:
X_train = X_train[used_columns]
X_test = X_test[used_columns]

<br>

`name`을 제외한 나머지는 수치형에 해당한다. `model_year`의 경우, 모델이 생산된 연도 정보이기 때문에 명목형으로 생각할 수 있지만,

여기서는 모델의 성능에 더 나중에 생산된 모델이 좋은 영향을 준다고 가정하고 수치형 변수로서 사용하려 한다.

결측치 처리 등의 전처리가 끝났으니 모델링을 진행한다.

<br><br>

### **데이터 모형 구축**

시작하기에 앞서 데이터 분할을 진행한다.

`train_test_split`를 사용해 학습 데이터와 검증 데이터로 다시 분류할 수 있다.

In [21]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

<br>

학습 데이터와 검증 데이터가 분할되었으니 수치형 변수들의 범위 차이에 따른 영향력이 왜곡되지 않도록

동일한 조건에서 분석을 진행하기 위해 Scaling을 진행한다.

<br>

`Randomforest`와 같은 모델의 경우에는 Scaling을 필수로 진행하지 않아도 큰 영향을 주지 않지만

보통은 Scaling을 하고 진행하는 것이 좋다는 점을 기억하도록 하자.

<br>

sklearn의 `preprocessing`모듈에서 `StandardScaler`를 import하여 Scaling을 진행하도록 하자.

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr)

X_tr = scaler.transform(X_tr)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

<br>

Scaling을 할 때 주의할 점이 있다면, 

**`X_train`에서 검증을 위해 나눈 `X_var`를 제외한 순수한 `X_tr`데이터만을 fit하여 `X_test`가 학습용 데이터에 어떠한 영향을 미치지 않도록 해야 한다.**

transform은 fit이 완료된 scaler를 순차적으로 사용하면 된다.

<br>

이제 준비가 되었으니 모델 학습을 진행한다.

sklearn의 `KNN`, `의사결정트리` 모델을 만들어보자.

<br>

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [35]:
print(KNeighborsClassifier.__doc__)

Classifier implementing the k-nearest neighbors vote.

    Read more in the :ref:`User Guide <classification>`.

    Parameters
    ----------
    n_neighbors : int, default=5
        Number of neighbors to use by default for :meth:`kneighbors` queries.

    weights : {'uniform', 'distance'}, callable or None, default='uniform'
        Weight function used in prediction.  Possible values:

        - 'uniform' : uniform weights.  All points in each neighborhood
          are weighted equally.
        - 'distance' : weight points by the inverse of their distance.
          in this case, closer neighbors of a query point will have a
          greater influence than neighbors which are further away.
        - [callable] : a user-defined function which accepts an
          array of distances, and returns an array of the same shape
          containing the weights.

    algorithm : {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'
        Algorithm used to compute the nearest neighbo

In [45]:
from sklearn.neighbors import KNeighborsClassifier

model_KNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
model_KNN.fit(X_tr, y_tr.values.squeeze())

KNeighborsClassifier(metric='euclidean')

In [54]:
from sklearn.tree import DecisionTreeClassifier

model_DT = DecisionTreeClassifier(max_depth=10)
model_DT.fit(X_tr, y_tr.values.squeeze())

DecisionTreeClassifier(max_depth=10)

<br><br>

### **데이터 모형 평가**

`fit`을 통해 학습이 완료된 모델에 대해 예측값을 구하거나, 예측 확률 값을 구할 수 있다.

이 문제의 경우 예측 확률 값을 구하는 것이 목적이기 때문에 `predict_proba` 함수를 사용한다.

In [55]:
y_val_pred_proba_KNN = model_KNN.predict_proba(X_val)
y_val_pred_proba_DT = model_DT.predict_proba(X_val)

In [59]:
from sklearn.metrics import roc_auc_score

score_KNN = roc_auc_score(y_val, y_val_pred_proba_KNN[:, 1])
score_DT = roc_auc_score(y_val, y_val_pred_proba_DT[:, 1])

print(score_KNN, score_DT)

0.8237492811960897 0.7883841288096607


<br>

나온 확률 값을 통해 `roc_auc_score` 함수로 AUROC 점수를 구할 수 있다.

사용한 KNN과 DT 중 KNN이 성능이 더 좋아 보인다.

<br>

이 방법 외에 하이퍼 파라미터를 직접 찾아 성능이 가장 좋은 모델을 선택할 수도 있다.


In [62]:
best_model = None
best_score = 0

for idx in range(2, 10):
    model = KNeighborsClassifier(n_neighbors=idx, metric='euclidean')
    model.fit(X_tr, y_tr.values.squeeze())
    
    y_val_pred_proba = model.predict_proba(X_val)
    score = roc_auc_score(y_val, y_val_pred_proba[:, 1])
    
    print(idx, "개의 이웃 확인 : ", score)
    if best_score <= score:
        best_model = model
        best_score = score

2 개의 이웃 확인 :  0.8225991949396205
3 개의 이웃 확인 :  0.8410005750431282
4 개의 이웃 확인 :  0.848188614146061
5 개의 이웃 확인 :  0.8237492811960897
6 개의 이웃 확인 :  0.8467510063254744
7 개의 이웃 확인 :  0.8579643473260494
8 개의 이웃 확인 :  0.8493387004025301
9 개의 이웃 확인 :  0.848188614146061


In [63]:
best_model

KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [64]:
best_model.predict_proba(X_test)

array([[0.71428571, 0.28571429],
       [0.28571429, 0.71428571],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.71428571, 0.28571429],
       [0.57142857, 0.42857143],
       [0.57142857, 0.42857143],
       [0.57142857, 0.42857143],
       [0.71428571, 0.28571429],
       [0.57142857, 0.42857143],
       [0.        , 1.        ],
       [0.42857143, 0.57142857],
       [0.        , 1.        ],
       [0.42857143, 0.57142857],
       [1.        , 0.        ],
       [0.85714286, 0.14285714],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.85714286, 0.14285714],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.57142857, 0.42857143],
       [0.71428571, 0.28571429],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.57142857, 0.42857143],
       [0.

<br>

미국에 해당하는 경우가 1이기 때문에 두번째 column에 해당하는 것이 추출해야하는 확률 값이다.


In [65]:
pred = best_model.predict_proba(X_test)[:, 1]
print(pred)

[0.28571429 0.71428571 1.         1.         0.28571429 0.42857143
 0.42857143 0.42857143 0.28571429 0.42857143 1.         0.57142857
 1.         0.57142857 0.         0.14285714 1.         1.
 0.14285714 1.         1.         1.         1.         1.
 1.         0.42857143 0.28571429 1.         1.         0.42857143
 0.14285714 0.         0.42857143 1.         1.         0.57142857
 1.         1.         0.14285714 0.71428571 1.         0.14285714
 0.         0.42857143 0.14285714 0.28571429 0.28571429 1.
 0.14285714 1.         0.         0.28571429 1.         0.14285714
 1.         1.         1.         0.85714286 1.         0.14285714
 1.         0.71428571 1.         1.         1.         1.
 1.         0.42857143 0.71428571 0.14285714 1.         1.
 0.71428571 0.         1.         0.42857143 1.         0.28571429
 0.57142857 1.         1.         1.         0.14285714 1.
 1.         1.         0.         0.14285714 0.14285714 1.
 1.         1.         1.         1.         1.    

<br><br>

결과물을 저장하도록 하자.

시험문제에서 대략적인 형태를 만들 수 있는 코드를 제공할 확률이 높으며,

해당 코드를 그대로 실행한다면 제출용 파일 저장이 가능하다.

In [66]:
pd.DataFrame({'isUSA': pred}).to_csv('./yemoonsaBigdata/res/003000000.csv', index=False)

시험에서는 하나의 모델을 먼저 완성한 다음에 그 결과물을 저장하는 것이 중요하므로,

성능이 떨어지는 단순한 기본 모형이라도 빠르게 완성한 뒤, 시간 여유가 있을 때 다양한 모형을 시험해보거나

하이퍼파라미터 튜닝을 하는 등의 작업이 필요하다는 것을 꼭 기억하도록 하자.